In [3]:
import pandas as pd
import json
from urllib.request import urlopen
import random
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

In [4]:

# FUNCTION THAT GIVES AVERAGE VALUE OF col_val IN col_group
def get_average_value_in_group_df(df, col_val, col_group):
    d = {col_group: df[col_group].unique()}
    for i, val in enumerate(col_val):
        data = list(df.groupby(col_group)[val].mean())
        d[val] = data
    new_df = pd.DataFrame(d)
    return new_df

def get_small_foncieres(lines):
    with open('valeursfoncieres-2022.txt', 'r') as f:
        d = f.readlines()
        with open('foncieres_small.txt', 'w') as f:
            f.write(d[0])
            for i in range(lines):
                index = random.randint(1, len(d)-1)
                f.write(d[index])


def get_departement_geojson_json_file():
    with urlopen('https://france-geojson.gregoiredavid.fr/repo/departements.geojson') as response:
        dep = json.load(response)
        dep_w = json.dumps(dep)
        dep_w = dep_w.replace('code', 'dep_code')
        with open('dep_geojson.json', 'w') as f:
            f.write(dep_w)

def get_regions_geojson_json_file():
    with urlopen('https://france-geojson.gregoiredavid.fr/repo/regions.geojson') as response:
        dep = json.load(response)
        dep_w = json.dumps(dep)
        dep_w = dep_w.replace('code', 'reg_code')
        with open('reg_geojson.json', 'w') as f:
            f.write(dep_w)

In [9]:
# Takes nbr_lines random lines in main file
#get_small_foncieres(100000)
# Get base data
data = pd.read_csv('foncieres_small.txt', sep='|', dtype={'Code departement': str})

FileNotFoundError: [Errno 2] No such file or directory: 'foncieres_small.txt'

In [ ]:
data["Surface terrain"] = data["Surface terrain"].fillna(0)  # NaN => 0
data["Valeur fonciere"] = data["Valeur fonciere"].str.replace(',', '.').astype(float)  # 380000,00 => 380000.0

In [ ]:
# Get dataframe with dep_code, terrain_area as columns, grouped by department codes
avg_terrain_area = get_average_value_in_group_df(data, col_val=['Surface terrain', 'Valeur fonciere'], col_group='Code departement')
# Plot the data departement-wise and change 'Surface terrain' label (default) to 'Surface terrain moyenne'
# Hover data adds data to what's shown when you hover over a departement

In [ ]:
def plot_data_choropleth_departements(df, value_compared, color_code='viridis', labels={}, hover_data=None):
    with open('dep_geojson.json', 'r') as f:
        dep_geo = json.load(f)
    fig = px.choropleth(df, geojson=dep_geo, color=value_compared, locations="Code departement",
                        featureidkey='properties.dep_code', color_continuous_scale=color_code,
                        labels=labels, hover_data=hover_data)
    fig.update_geos(visible=False, fitbounds='locations')
    return fig

In [ ]:
def plot_data_choropleth_region(df, value_compared, color_code='viridis', labels=None):
    with open('reg_geojson.json.json', 'r') as f:
        dep_geo = json.load(f)
    fig = px.choropleth(df, geojson=dep_geo, color=value_compared, locations="Code region",
                        featureidkey='properties.reg_code', color_continuous_scale=color_code,
                        labels=labels if labels is not None else {})
    fig.update_geos(visible=False, fitbounds='locations')
    return fig

In [ ]:
fig = plot_data_choropleth_departements(avg_terrain_area, value_compared='Valeur fonciere',
                                        labels={'Surface terrain' : 'Surface terrain moyenne', 'Valeur fonciere': 'VF moyenne'},
                                        hover_data=['Surface terrain'])
fig.show()